In [ ]:
# !git clone --quiet --recursive https://github.com/cvg/Hierarchical-Localization/
%cd Hierarchical-Localization
# !pip install --progress-bar off --quiet -e .
# !pip install --progress-bar off --quiet --upgrade plotly

import tqdm, tqdm.notebook
tqdm.tqdm = tqdm.notebook.tqdm  # notebook-friendly progress bars
from pathlib import Path

from hloc import extract_features, match_features, reconstruction, visualization, pairs_from_exhaustive, semantic_features
from hloc.visualization import plot_images, read_image
from hloc.utils import viz_3d

In [ ]:
images = Path('datasets/sacre_coeur')
images2 = Path('datasets/sacre_coeur/seg_mapping2/')
outputs = Path('outputs/demo1/')
outputs_h5 = Path('outputs/semantic_demo.h5')
!rm -rf $outputs
!rm -rf $outputs2

# SfM에 사용될 이미지 페어 파일 경로와 로컬라이제이션에 사용될 이미지 페어 파일 경로 설정
sfm_pairs = outputs / 'pairs-sfm.txt'
loc_pairs = outputs / 'pairs-loc.txt'
sfm_dir = outputs / 'sfm'
# 특징과 매칭 결과를 저장할 파일 경로 설정
features = outputs / 'features.h5'
matches = outputs / 'matches.h5'
# 특징 추출 및 매칭 설정 가져오기
feature_conf = extract_features.confs['disk']
matcher_conf = match_features.confs['disk+lightglue']

In [ ]:
# 참조 이미지 목록 설정
references = [str(p.relative_to(images)) for p in (images / 'mapping2/').iterdir() if not p.name.startswith('.')]
print(len(references), "mapping images")
# 참조 이미지들 플롯
plot_images([read_image(images / r) for r in references], dpi=25)

In [ ]:
references2 = [str(p.relative_to(images2)) for p in (images2 / 'seg_mapping2/').iterdir() if not p.name.startswith('.')]
print(len(references2), "segmenation mapping images")
plot_images([read_image(images2 / r) for r in references2], dpi=25)

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
# 시맨틱 세그멘테이션 정보 추출
semantic_features.extract_semantic_segmentation(images2, outputs_h5)
semantic_features.extract_features(images2, outputs_h5)
# 이미지에서 특징 추출
extract_features.main(feature_conf, images, image_list=references, feature_path=features)
# SfM에 사용할 이미지 페어 목록 생성
pairs_from_exhaustive.main(sfm_pairs, image_list=references)
# 이미지 특징 매칭 수행
match_features.main(matcher_conf, sfm_pairs, features=features, matches=matches)

In [ ]:
# 3D 재구성 수행
model = reconstruction.main(sfm_dir, images, sfm_pairs, features, matches, image_list=references)

In [ ]:
fig = viz_3d.init_figure()
viz_3d.plot_reconstruction(fig, model, color='rgba(255,0,0,0.5)', name="mapping", points_rgb=True)
fig.show()

In [ ]:
query = 'query.jpg'
!mkdir -p $images/query && $images/$query -q
plot_images([read_image(images / query)], dpi=75)

In [ ]:
extract_features.main(feature_conf, images, image_list=[query], feature_path=features, overwrite=True)
pairs_from_exhaustive.main(loc_pairs, image_list=[query], ref_list=references)
match_features.main(matcher_conf, loc_pairs, features=features, matches=matches, overwrite=True);

In [ ]:
import pycolmap
from hloc.localize_sfm import QueryLocalizer, pose_from_cluster
# 로컬라이제이션에 사용할 이미지와 관련 설정 가져오기
camera = pycolmap.infer_camera_from_image(images / query)
ref_ids = []
# 이미지 모델에서 참조 이미지의 ID 가져오기
for r in references:
    image_info = model.find_image_with_name(r)
    if image_info is not None:
        ref_ids.append(image_info.image_id)
    else:
        print(f"Image '{r}' not found in the model.")
# 로컬라이제이션을 위한 설정 정의
conf = {
    'estimation': {'ransac': {'max_error': 12}},
    'refinement': {'refine_focal_length': True, 'refine_extra_params': True},
}
# QueryLocalizaer 초기화
localizer = QueryLocalizer(model, conf)
# 이미지 로컬라이제이션을 수행하고 결과 가져오기
ret, log = pose_from_cluster(localizer, query, camera, ref_ids, features, matches)
# 일치하는 correspondence 수 및 결과 시각화
print(f'found {ret["num_inliers"]}/{len(ret["inliers"])} inlier correspondences.')
visualization.visualize_loc_from_log(images, query, log, model)

In [ ]:
import numpy as np
pose = pycolmap.Image(tvec=ret['tvec'], qvec=ret['qvec'])
viz_3d.plot_camera_colmap(fig, pose, camera, color='rgba(0,255,0,0.5)', name=query, fill=True)
# visualize 2D-3D correspodences
inl_3d = np.array([model.points3D[pid].xyz for pid in np.array(log['points3D_ids'])[ret['inliers']]])
viz_3d.plot_points(fig, inl_3d, color="lime", ps=1, name=query)
fig.show()